# I - Analyse exploratoire

Ce notebook suit le flux recommandé : chargement des données, statistiques descriptives, puis synthèse des enseignements.


## 1. Analyses statistiques descriptives univariées

Nous parcourons les variables une à une pour identifier les déséquilibres et guider la préparation des données.


### a. Fréquences des variables

Les distributions sont présentées de manière tronquée (`head()`) afin de conserver des sorties compactes.


In [1]:
import io
from pathlib import Path
import pandas as pd


In [2]:
from pathlib import Path


def find_project_root(start: Path) -> Path:
    """Climb up until a directory containing 'data' exists (max 6 levels)."""
    current = start
    for _ in range(6):
        if (current / 'data').exists():
            return current
        if current.parent == current:
            break
        current = current.parent
    return start

PROJECT_ROOT = find_project_root(Path.cwd())
file_path = PROJECT_ROOT / 'data' / 'dataset.csv'
print(f"Chemin du fichier détecté: {file_path}")


Chemin du fichier détecté: /Users/steven/Documents/Programmation/MentalHealth_CatBoost/data/dataset.csv


Chargement du jeu de données et présentation d'un aperçu contrôlé pour éviter les sorties trop volumineuses.


In [3]:
def load_data(max_lignes=5, max_statistiques=10):
    """Charge le dataset et affiche un aperçu limité pour rester lisible."""
    df = pd.read_csv(file_path)
    print("Aperçu des premières lignes :")
    print(df.head(max_lignes))
    print("\nRésumé d'information :")
    buffer = io.StringIO()
    df.info(buf=buffer)
    print(buffer.getvalue())
    print("\nStatistiques descriptives (aperçu) :")
    stats = df.describe(include='all').transpose().head(max_statistiques)
    print(stats)
    return df


df = load_data()


Aperçu des premières lignes :
   Gender Country Occupation SelfEmployed FamilyHistory Treatment  \
0  Female      UK     Others           No           Yes        No   
1  Female     USA  Housewife           No           Yes        No   
2  Female  Canada     Others           No            No       Yes   
3  Female  Canada  Corporate           No           Yes       Yes   
4  Female      UK  Corporate           No            No       Yes   

          DaysIndoors HabitsChange MentalHealthHistory IncreasingStress  \
0          15-30 days           No                 Yes              Yes   
1          15-30 days        Maybe               Maybe              Yes   
2  More than 2 months        Maybe                  No               No   
3    Go out Every day           No               Maybe               No   
4  More than 2 months          Yes                  No            Maybe   

  MoodSwings SocialWeakness CopingStruggles WorkInterest SocialWeakness.1  \
0       High             No

In [4]:
def frequences_absolues_relatives(df, max_modalites=8):
    """Affiche les fréquences absolues et relatives avec un aperçu limité pour chaque colonne catégorielle."""
    print('=' * 80 + "\nANALYSE DES FRÉQUENCES - TOUTES LES VARIABLES\n" + '=' * 80)
    resultats = {}
    for col in df.columns:
        print(f"\n{'=' * 80}\nVariable : {col}\n{'=' * 80}")
        freq_abs = df[col].value_counts().sort_index()
        freq_rel = (df[col].value_counts(normalize=True).sort_index() * 100).round(2)
        table_complete = pd.concat([
            pd.DataFrame({'Fréquence absolue': freq_abs, 'Fréquence relative (%)': freq_rel}),
            pd.DataFrame({'Fréquence absolue': [freq_abs.sum()], 'Fréquence relative (%)': [100.0]}, index=['TOTAL'])
        ])
        apercu = pd.concat([table_complete.head(max_modalites - 1), table_complete.tail(1)]).drop_duplicates()
        print(apercu)
        lignes_cachees = max(len(table_complete) - max_modalites, 0)
        if lignes_cachees > 0:
            print(f"... ({lignes_cachees} modalités supplémentaires masquées)")
        print(f"\nNombre de catégories : {len(freq_abs)}")
        resultats[col] = {'freq_abs': freq_abs, 'freq_rel': freq_rel, 'nb_cat': len(freq_abs)}
    print("\n" + "=" * 80 + "\nANALYSE TERMINÉE\n" + "=" * 80)
    return resultats


resultats_freq = frequences_absolues_relatives(df)


ANALYSE DES FRÉQUENCES - TOUTES LES VARIABLES

Variable : Gender
        Fréquence absolue  Fréquence relative (%)
Female              30714                   11.75
Male               230614                   88.25
TOTAL              261328                  100.00

Nombre de catégories : 2

Variable : Country
                        Fréquence absolue  Fréquence relative (%)
Australia                            3544                    1.36
Belgium                               477                    0.18
Bosnia and Herzegovina                237                    0.09
Brazil                               1338                    0.51
Canada                              14177                    5.42
Colombia                              384                    0.15
TOTAL                              261328                  100.00
... (28 modalités supplémentaires masquées)

Nombre de catégories : 35

Variable : Occupation
           Fréquence absolue  Fréquence relative (%)
Business      

- Surreprésentation des hommes dans le jeu de données.
- Trois pays dominent : États-Unis, Royaume-Uni et Canada ; regrouper les autres pays peut être pertinent.
- La variable `self-employed` est peu diversifiée mais peut rester informative.
- La cible `treatment` est parfaitement répartie entre oui et non.
- Peu de personnes (≈3 %) déclarent avoir consulté pour des raisons de santé mentale.


### b. Mode

Nous calculons la modalité la plus fréquente pour chaque variable afin de préparer d'éventuels imputations simples.


In [5]:
def calculer_mode(df):
    """Calcule et affiche le mode (catégorie la plus fréquente) pour chaque variable."""
    print('=' * 80 + "\nANALYSE DU MODE - TOUTES LES VARIABLES\n" + '=' * 80)
    modes_data = []
    for col in df.columns:
        mode_val = df[col].mode()[0]
        freq = df[col].value_counts()[mode_val]
        pct = round((freq / len(df)) * 100, 2)
        modes_data.append({'Variable': col, 'Mode': mode_val, 'Fréquence': freq, 'Pourcentage (%)': pct})
    modes_df = pd.DataFrame(modes_data)
    print(modes_df.to_string(index=False))
    print("\n" + "=" * 80 + "\nANALYSE TERMINÉE\n" + "=" * 80)
    return {row['Variable']: {'mode': row['Mode'], 'freq': row['Fréquence'], 'pct': row['Pourcentage (%)']}
            for _, row in modes_df.iterrows()}


resultats_mode = calculer_mode(df)


ANALYSE DU MODE - TOUTES LES VARIABLES
             Variable      Mode  Fréquence  Pourcentage (%)
               Gender      Male     230614            88.25
              Country       USA     154478            59.11
           Occupation Housewife      60081            22.99
         SelfEmployed        No     230404            88.17
        FamilyHistory        No     161981            61.98
            Treatment        No     132958            50.88
          DaysIndoors 1-14 days      57598            22.04
         HabitsChange       Yes      97687            37.38
  MentalHealthHistory        No      94027            35.98
     IncreasingStress     Maybe      90697            34.71
           MoodSwings    Medium      90215            34.52
       SocialWeakness     Maybe      93271            35.69
      CopingStruggles        No     139251            53.29
         WorkInterest        No      94460            36.15
     SocialWeakness.1     Maybe      93271            35.69
M

### c. Nombre de valeurs uniques par variable

Cette mesure permet de repérer les variables quasi constantes ou, au contraire, très hétérogènes.


In [6]:
def valeurs_uniques(df):
    """Affiche le nombre de valeurs uniques pour chaque variable."""
    print('=' * 80 + "\nNOMBRE DE VALEURS UNIQUES PAR VARIABLE\n" + '=' * 80)
    uniques_data = [{'Variable': col, 'Valeurs uniques': df[col].nunique()} for col in df.columns]
    uniques_df = pd.DataFrame(uniques_data)
    print(uniques_df.to_string(index=False))
    print("\n" + "=" * 80 + "\nANALYSE TERMINÉE\n" + "=" * 80)
    return {row['Variable']: row['Valeurs uniques'] for _, row in uniques_df.iterrows()}


resultats_uniques = valeurs_uniques(df)


NOMBRE DE VALEURS UNIQUES PAR VARIABLE
             Variable  Valeurs uniques
               Gender                2
              Country               35
           Occupation                5
         SelfEmployed                2
        FamilyHistory                2
            Treatment                2
          DaysIndoors                5
         HabitsChange                3
  MentalHealthHistory                3
     IncreasingStress                3
           MoodSwings                3
       SocialWeakness                3
      CopingStruggles                2
         WorkInterest                3
     SocialWeakness.1                3
MentalHealthInterview                3
          CareOptions                3

ANALYSE TERMINÉE


Une fois la variable `Country` regroupée, le bruit devrait diminuer nettement lors des phases de modélisation.
